In [ ]:
!pip install --upgrade transformers accelerate bitsandbytes datasets sentencepiece huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)


CUDA available: True
GPU Name: NVIDIA A100-SXM4-40GB
CUDA version: 12.4


In [ ]:
from datasets import load_dataset
dataset = load_dataset("jkhedri/psychology-dataset")
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)05-b938-3729b352d322-c000.snappy.parquet:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

(…)05-b938-3729b352d322-c000.snappy.parquet:   0%|          | 0.00/96.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'response_j', 'response_k'],
        num_rows: 9846
    })
})


In [ ]:
from transformers import pipeline
paraphraser = pipeline("text2text-generation", model="t5-base", tokenizer="t5-base", device=0)

def process_sample(example):
    sample_j = {
        "prompt": example["question"] + "\nThis is what you should",
        "response": example["response_j"]
    }
    paraphrase_input = "paraphrase: " + example["response_k"] + " </s>"
    paraphrased = paraphraser(paraphrase_input, max_length=256, num_beams=5, early_stopping=True)[0]['generated_text']
    sample_k = {
        "prompt": example["question"] + "\nDo not think negatively in this way",
        "response": "Remember: " + paraphrased
    }
    return [sample_j, sample_k]

processed_samples = []
for example in dataset["train"]:
    processed_samples.extend(process_sample(example))

from datasets import Dataset
train_dataset = Dataset.from_list(processed_samples)
print("Number of training samples:", len(train_dataset))


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Number of training samples: 19692


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)


from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("Quantized model loaded and LoRA applied.")


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Quantized model loaded and LoRA applied.


In [ ]:
def tokenize_function(example):
    combined = [p + tokenizer.eos_token + r for p, r in zip(example["prompt"], example["response"])]
    tokenized = tokenizer(combined, truncation=True, max_length=512)
    prompt_lengths = [len(tokenizer(p, truncation=True, max_length=512)["input_ids"]) for p in example["prompt"]]

    labels = [list(ids) for ids in tokenized["input_ids"]]
    for i, pl in enumerate(prompt_lengths):
        labels[i][:pl] = [-100] * pl

    tokenized["labels"] = labels
    return tokenized
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
print("Tokenization complete.")
sample = tokenized_dataset[0]
print("Input IDs:", sample["input_ids"])
print("Labels:", sample["labels"])


Map:   0%|          | 0/19692 [00:00<?, ? examples/s]

Tokenization complete.
Input IDs: [151646, 40, 2776, 8266, 2167, 37000, 30345, 323, 358, 1513, 944, 1414, 3170, 624, 1986, 374, 1128, 498, 1265, 151643, 2132, 594, 4185, 311, 2666, 37000, 518, 3039, 11, 323, 1052, 646, 387, 1657, 7966, 369, 432, 13, 12243, 1052, 1012, 894, 3213, 4344, 476, 8464, 1087, 304, 697, 2272, 429, 1231, 387, 28720, 311, 697, 18056, 30, 6771, 594, 975, 3786, 311, 10542, 894, 30754, 323, 2225, 61982, 14830, 311, 10091, 697, 18056, 13]
Labels: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 151643, 2132, 594, 4185, 311, 2666, 37000, 518, 3039, 11, 323, 1052, 646, 387, 1657, 7966, 369, 432, 13, 12243, 1052, 1012, 894, 3213, 4344, 476, 8464, 1087, 304, 697, 2272, 429, 1231, 387, 28720, 311, 697, 18056, 30, 6771, 594, 975, 3786, 311, 10542, 894, 30754, 323, 2225, 61982, 14830, 311, 10091, 697, 18056, 13]


In [ ]:
import torch

def custom_data_collator(features):
    labels = [feature.pop("labels") for feature in features]
    batch = tokenizer.pad(features, padding=True, return_tensors="pt")
    max_label_len = max(len(label) for label in labels)
    padded_labels = [label + [-100] * (max_label_len - len(label)) for label in labels]
    batch["labels"] = torch.tensor(padded_labels)

    return batch


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen_lora_finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_steps=200,
    optim="adamw_torch",
    evaluation_strategy="no",
    save_total_limit=3,
    dataloader_num_workers=0,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=custom_data_collator
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
batch = custom_data_collator([tokenized_dataset[0]])
batch = {k: v.to(model.device) for k, v in batch.items()}

outputs = model(**batch)
print("Forward pass loss:", outputs.loss)


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Forward pass loss: tensor(2.6868, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
trainer.train()


Step,Training Loss
50,7.065700
100,4.701000
150,4.522000
200,4.338300
250,4.277300
300,4.190700
350,4.112900
400,4.079900
450,4.016300
500,4.008100


TrainOutput(global_step=1845, training_loss=3.978794691297743, metrics={'train_runtime': 2391.3115, 'train_samples_per_second': 24.704, 'train_steps_per_second': 0.772, 'total_flos': 4.995161527775232e+16, 'train_loss': 3.978794691297743, 'epoch': 2.9959382615759544})

In [ ]:
model.eval()

sample_prompts = [
    "How can I improve my mental health?\nThis is what you should",
    "What do I do when I'm feeling anxious?\nDo not think negatively in this way"
]

for prompt in sample_prompts:
    combined_input = prompt + tokenizer.eos_token
    inputs = tokenizer(combined_input, return_tensors="pt").to(model.device)

    outputs = model.generate(
    **inputs,
    max_length=256,
    do_sample=True,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2,
    temperature=0.8
)


    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Prompt:")
    print(prompt)
    print("\nGenerated Response:")
    if generated_text.startswith(prompt):
        generated_response = generated_text[len(prompt):].strip()
    else:
        generated_response = generated_text
    print(generated_response)
    print("="*50)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Prompt:
How can I improve my mental health?
This is what you should

Generated Response:
it, important to for. have help the your's issues together any in socializing or may include therapy self and medication treatment a or depression? develop coping strategies like seeking ways explore different activities that create feel helps set a schedule regular routine. It.  Additionally: support work with your friends or family provide resources such as counseling services access to affordable find also options meditation technique lifestyle. Your own unique individual needs. For example sleep on healthy about exercise physical activity take breaks after working be more productive of time better mood. Remembering your worth by making sure things are not too difficult feeling good would increase satisfaction confidence overall well life quality You could talk therapist if needed. If these all aren't helping plan to look into alternative approaches issue such underlying factors contributing may